# Pathway

# Developing a Project Focus



## Starting with Smart Yields | Moving Toward Citrus

### Weather Anomalies Over Time
We started analyzing the Smart Yields dataset with an initial interest in analyzing the effect weather anomalies have on crop yields over time after learning about the work of Vincent Kimura and Axel Holland for their startup. In an attempt to help them build stronger models, we wanted to provide insights into what variables may be important indicators in idenfitying impending frost. However, we realized that after plotting the Smart Yields time series, the data only spanned from March 27th of this year until May 8th. We needed to augment the data to develop a more robust dataset, which would give us more opportunity for using more techniques. 

Smart Yields included data on apples. Fruit trees are particularly sensitive to fluctuations in weather, so we decided that choosing fruit tree crops, in particular, might show a relatively high amount of variation and make for an interesting project. We knew we wanted to analyze the impact of weather on fruit trees over time.

### The appeal of citrus
For ease of modeling, we simplified it one step further and selected citrus. The three broad categories of citrus crops that we decided to focus on were oranges, grapefruit, and lemons. This choice was was primarily due to the data that was available to us.

We decided that citrus trees would make for an interesting study crop, as citrus trees are susceptible to extremes in temperature, since because they do not hibernate or shed leaves in winter. Also, per capita consumption of oranges is higher than that any other fruit. Finally, the citrus industry has large economic contributions. For instance, it adds an annual seven billion dollars to the California economy alone.

### Selecting California
Due to this economic impact in California, as well as the variety and volume of citrus produced in the state, we decided to focus on California as a case study for our exploration of the effects of weather on citrus production. California is also one of the top four citrus producing states in the nation, alongside Texas, Arizona, and Florida. 

The following pie charts break down the production by percentage for oranges, grapefruit, and lemons by state:

<div style="display: flex;">
<img style= "float: left;" src="lemonspie.jpg" height="400" width="400"> 
<img style= "float: center;" src="grapefruitpie.jpg" height="400" width="400"> 
<img style= "float: right;" src="orangespie.jpg" height="400" width="400">
<div/>



# Data Acquisition

# Identifying Questions

## Noticing Trends in Texas | How are crops affected by freezing temperatures?
We first noticed a delay between freeze event occurrences and reported bearing acreage in Texas in 1984. This pattern was seen once again with grapefruit and oranges between 1989 and 1990. Therefore, for analyzing the causative factors behind the trends of bearing acreage over time, it seemed important to look at extreme temperature lows that occurred during the previous season. 

An article from [the Texas Agricultural Extension Service](https://aggie-horticulture.tamu.edu/citrus/l2319.htm) mirrored our findings, further highlighting the severity of crop loss experienced in the succeeding seasons after the frosts. 


# Methods | Building Models

## Equations in LaTex
## Linear Regression
For our modeling, we wanted to start with a big picture overview before deep diving into the granular elements, using scatterplots and linear regression of citrus bearing acreage with aggregated totals in the United States to start initital analysis.

*Practical Statistics for Data Scientists* emphasizes “by itself, the regression equation does not prove the direction of causation. Conclusions about causation must come from a broader context of understanding about the relationship.” For instance, nationally, there has been a noticeable decrease in bearing acreage for oranges, grapefruit, and lemons. in the past twenty years with a predicted decrease in the next five years. This answers the “what,” but further exploration of correlation is needed to discover the possibilities in the “why.”

<img src="bearingacreage.png">



# Analyzing Data

# Validating and Evaluating the Analysis and Models

# Challenges | New Questions

# Future Studies